In [24]:
import os, glob, random
import pandas as pd
import SimpleITK as sitk
import numpy as np

def get_image_path_by_id(patient_id, image_dir):
    image_order  = patient_id
    file_name = [os.path.relpath(os.path.join(image_dir, x)) \
                    for x in os.listdir(image_dir) \
                    if os.path.isfile(os.path.join(image_dir, x)) and patient_id in x][0] 
    return file_name

In [ ]:
df_init = pd.DataFrame()


manual_seg_dir =  'nnUNet_raw_data_base/nnUNet_test_data/test_seg_in_nii/'
auto_seg_dir =  'nnUNet_raw_data_base/nnUNet_test_data/test_label_in_nii/'
csv_write_path= 'nnUNet_raw_data_base/nnUNet_test_data/MRI_volume_internal_test331.csv'


for idx, d in sorted(enumerate(os.listdir(manual_seg_dir))):
    if '.nii.gz' in d :
        print(d,'processing')

        indices = [i for i, c in enumerate(d) if c == '_']
        pid = d[:indices[1]]
        if len(indices)>2 and ('Bd' in d or 'Melb' in d ):
            pid = d[:indices[2]]     
        
        auto_seg_path = get_image_path_by_id(pid, auto_seg_dir)
        manual_seg_path = get_image_path_by_id(pid, manual_seg_dir)

        if os.path.exists(auto_seg_path) and os.path.exists(manual_seg_path)  :
            manual_seg_sitk = sitk.ReadImage(manual_seg_path)
            mskCube  = sitk.GetArrayFromImage(manual_seg_sitk)
            auto_seg_sitk = sitk.ReadImage(auto_seg_path)
            prdCube = sitk.GetArrayFromImage(auto_seg_sitk)
            
            mskCube1 = np.copy(mskCube)
            mskCube1[mskCube1!=1]=0      
            prdCube1 = np.copy(prdCube)
            prdCube1[prdCube1!=1]=0

            mskCube2 = np.copy(mskCube)
            mskCube2[mskCube2!=2]=0
            prdCube2 = np.copy(prdCube)
            prdCube2[prdCube2!=2]=0

            mskCube3 = np.copy(mskCube)
            mskCube3[mskCube3!=3]=0
            prdCube3 = np.copy(prdCube)
            prdCube3[prdCube3!=3]=0

            mskCube4 = np.copy(mskCube)
            mskCube4[mskCube4!=4]=0
            prdCube4 = np.copy(prdCube)
            prdCube4[prdCube4!=4]=0
            
            spacing = manual_seg_sitk.GetSpacing()
    
            volume1_gd = np.sum(mskCube1) *spacing[0] *spacing[1] *spacing[2] /1000
            volume1_ai = np.sum(prdCube1) *spacing[0] *spacing[1] *spacing[2]  /1000         
            volume2_gd = np.sum(mskCube2) *spacing[0] *spacing[1] *spacing[2] /1000 / 2
            volume2_ai = np.sum(prdCube2) *spacing[0] *spacing[1] *spacing[2] /1000  / 2
            
            volume3_gd = np.sum(mskCube3)  *spacing[0] *spacing[1] *spacing[2] /1000 / 3
            volume3_ai = np.sum(prdCube3)  *spacing[0] *spacing[1] *spacing[2]/1000   / 3      
            volume4_gd = np.sum(mskCube4) *spacing[0] *spacing[1] *spacing[2]/1000 / 4
            volume4_ai = np.sum(prdCube4) *spacing[0] *spacing[1] *spacing[2] /1000  / 4        
           

            df = pd.DataFrame({'pid':  pid,
                               
                        'volume1_gd':round(volume1_gd,5), 'volume1_ai':round(volume1_ai,5),
                        'volume2_gd':round(volume2_gd,5), 'volume2_ai':round(volume2_ai,5),
                               'volume3_gd':round(volume3_gd,5), 'volume3_ai':round(volume3_ai,5),
                               'volume4_gd':round(volume4_gd,5), 'volume4_ai':round(volume4_ai,5)
                                                 
                            },index=[0])
            df_init = df_init.append(df)
            df_init.to_csv(csv_write_path )
            print(idx,'th',pid,' done \n\n')
